In [1]:
#@markdown Configuración Inicial
from IPython.utils import io

from google.colab.data_table import DataTable
from IPython.display import Javascript
from IPython.display import display, Markdown, Latex
from IPython.display import Audio, Image
from IPython.display import IFrame, HTML
with io.capture_output() as capt:
  !pip install gradio

import gradio as gr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

def dLatex(self):
  return display(Latex(self))

def dMarkdown(self):
  return display(Markdown(self))

# Avoids scroll-in-the-scroll in the entire Notebook
# https://stackoverflow.com/a/66891328
def resize_colab_cell():
  display(Javascript(
      'google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'
      ))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

def dLatex(self):
  return display(Latex(self))

def dMarkdown(self):
  return display(Markdown(self))

# Formula Somigliana

\begin{align}
\gamma_0(\varphi)
&=
\frac{
  a\gamma_a\cos^2(\varphi)
  +
  b
  \gamma_b\sin^2(\varphi)
  }{
  \sqrt{
    a^2\cos^2(\varphi)
    +
    b^2\sin^2(\varphi)
  }
}
\\
&=
\gamma_a
\frac{
  1+
  p\sin^2(\varphi)
  }{
  \sqrt{
    1-
    e^2\sin^2(\varphi)
  }
}
\end{align}

* $a$: Radio Ecuatorial
* $b$: Radio Polar
* $\gamma_a$: Gravedad Ecuatorial
* $\gamma_b$: Gravedad Polar
* $\varphi$: Latitud


\begin{align}
p 
&=
\frac{b\,\gamma_b}{a\,\gamma_a} - 1
\\
e^2
&=
1 - \frac{b^2}{a^2}
\end{align}

In [2]:
#@markdown Según el elipsoide de referencia GSR80:
#@markdown \begin{align}
#@markdown a&=6\,378.137\,\phantom{0000}\,\mathrm{km}
#@markdown &
#@markdown \gamma_a&=978.032\,677\,15\,\mathrm{Gal}
#@markdown \\
#@markdown b&=6\,356.752\,3141\, \mathrm{km}
#@markdown &
#@markdown \gamma_b&=983.218\,636\,85\, \mathrm{Gal}
#@markdown \end{align}

GSR80 = {
    'ra': 6_378.137,
    'rb': 6_356.752_3141,
    'ya': 978.032_677_15,
    'yb': 983.218_636_85,
    'c_list': [5.279_0414e-3, 2.327_18e-5, 1.262e-7, 7e-10]
    }

def Somigliana(lat, ellipsoid=GSR80):
  phi = np.radians(lat)
  ra, rb = [GSR80[key] for key in ('ra','rb')]
  ya, yb = [GSR80[key] for key in ('ya','yb')]
  Nu = ra*ya*np.cos(phi)**2 +\
       rb*yb*np.sin(phi)**2
  De = np.sqrt(
       ra**2*np.cos(phi)**2 +\
       rb**2*np.sin(phi)**2)
  gamma_0 = Nu/De
  return gamma_0

<IPython.core.display.Javascript object>

In [3]:
#@markdown Para los siguientes se usa una aproximación del tipo
#@markdown \begin{align}
#@markdown \gamma_0(\varphi)
#@markdown =
#@markdown \gamma_a
#@markdown (1 + \beta_0\sin^2 \varphi + \beta_1\sin^2 2\varphi + \cdots)
#@markdown \end{align}

def Int_gravity(lat, parameters):
  phi = np.radians(lat)
  ya   = parameters['ya']
  beta = parameters['beta']
  gamma_0 = ya*(1 +\
                beta[0]*np.sin(phi)**2 +\
                beta[1]*np.sin(2*phi)**2)
  return gamma_0

<IPython.core.display.Javascript object>

In [4]:
#@title 1930
#@markdown $$\gamma_a = 978.049\,\mathrm{Gal} \quad \beta_0 = 5.2884 \cdot 10^{-3} \quad \beta_1 = -5.9 \cdot 10^{-6}$$

par1930 = {
    'ya': 978.049,
    'beta': [5.2884e-3, -5.9e-6]
}

<IPython.core.display.Javascript object>

In [5]:
#@title 1948
#@markdown $$\gamma_a = 978.0373\,\mathrm{Gal} \quad \beta_0 = 5.2891 \cdot 10^{-3} \quad \beta_1 = -5.9 \cdot 10^{-6}$$

par1948 = {
    'ya': 978.0373,
    'beta': [5.2891e-3, -5.9e-6]
}

<IPython.core.display.Javascript object>

In [6]:
#@title 1967
#@markdown $$\gamma_a = 978.0318\,\mathrm{Gal} \quad \beta_0 = 5.3024 \cdot 10^{-3} \quad \beta_1 = -5.9 \cdot 10^{-6}$$


par1967 = {
    'ya': 978.0318,
    'beta': [5.3024e-3, -5.9e-6]
}

<IPython.core.display.Javascript object>

In [7]:
#@title 1980
#@markdown $$\gamma_a = 978.0327,\mathrm{Gal} \quad \beta_0 = 5.3024 \cdot 10^{-3} \quad \beta_1 = -5.8 \cdot 10^{-6}$$

par1980 = {
    'ya': 978.0327,
    'beta': [5.3024e-3, -5.8e-6]
}

<IPython.core.display.Javascript object>

In [8]:
#@title GRS-1980
#@markdown $$\gamma_0(\varphi) = \gamma_a (1 + c_1\sin^2 \varphi + c_2\sin^4 \varphi + c_3\sin^6 \varphi + c_4\sin^8\varphi + \cdots)$$
#@markdown 
#@markdown Donde
#@markdown 
#@markdown * $\gamma_a = 978.032\,677\,15\,\mathrm{Gal}$
#@markdown * $c_0 = 5.279\,0414\cdot{10}^{-3}$
#@markdown * $c_1 = 2.327\,18\cdot{10}^{-5}$
#@markdown * $c_2 = 1.262\cdot{10}^{-7}$
#@markdown * $c_3 = 7\cdot{10}^{-10}$

def GRS_grav(lat, ellipsoid=GSR80):
  phi = np.radians(lat)
  ya = ellipsoid['ya']
  c  = ellipsoid['c_list']
  gamma_0 = ya*(1 +\
                c[0]*np.sin(phi)**2 +\
                c[1]*np.sin(phi)**4 +\
                c[2]*np.sin(phi)**6 +\
                c[3]*np.sin(phi)**8 )
  return gamma_0

<IPython.core.display.Javascript object>

In [9]:
#@title 1984
#@markdown \begin{align}
#@markdown g_{0}
#@markdown =
#@markdown \gamma_a
#@markdown \frac{
#@markdown   1+p\sin^{2}(\varphi)
#@markdown }{
#@markdown   \sqrt{1-e^2\sin^{2}(2\varphi)}
#@markdown }
#@markdown \end{align}

#@markdown Donde:
#@markdown * $\gamma_a=978.032\,677\,14\,\mathrm{Gal}$
#@markdown * $p\phantom{^2} = 0.001\,931\,851\,386\,39$
#@markdown * $e^2 = 0.006\,694\,379\,990\,13$

par1984 = {
    'ya': 978.032_677_14,
    'p' : 0.001_931_851_386_39,
    'e2': 0.006_694_379_990_13
}

def g1984(lat, par):
  phi = np.radians(lat)
  ya, p, e2 = [par[key] for key in ('ya', 'p', 'e2')]
  Nu = 1 + p*np.sin(phi)**2
  De = np.sqrt(1-e2*np.sin(phi)**2)
  gamma_0 = ya*Nu/De
  return gamma_0

<IPython.core.display.Javascript object>

In [10]:
#@title Tabla
def dic_grav(lat):
  dic = {
      'Formula Somigliana'  : Somigliana(lat, GSR80),
      'Internacional 1930'  : Int_gravity(lat, par1930),
      'Internacional 1948'  : Int_gravity(lat, par1948),
      'Internacional 1967'  : Int_gravity(lat, par1967),
      'Internacional 1980_1': Int_gravity(lat, par1980),
      'Internacional 1980_2': GRS_grav(lat, GSR80),
      'Internacional 1984'  : g1984(lat, par1984),
      }
  return dic

lats = np.linspace(-90, +90, 181, endpoint=True, dtype='int')
data=np.vectorize(dic_grav)(lats)
df = pd.DataFrame(list(data), index=lats)
df.index.name='Lat'
display(DataTable(df, num_rows_per_page=10))

<IPython.core.display.Javascript object>

,Formula Somigliana,Internacional 1930,Internacional 1948,Internacional 1967,Internacional 1980_1,Internacional 1980_2,Internacional 1984
Lat,,,,,,,
-90,983.218637,983.221314,983.210237,983.217716,983.218621,983.218637,983.218637
-89,983.217050,983.219732,983.208654,983.216129,983.217034,983.217050,983.217050
-88,983.212292,983.214987,983.203908,983.211371,983.212277,983.212292,983.212292
-87,983.204369,983.207084,983.196005,983.203448,983.204354,983.204369,983.204369
-86,983.193291,983.196034,983.184954,983.192370,983.193276,983.193291,983.193291
...,...,...,...,...,...,...,...
86,983.193291,983.196034,983.184954,983.192370,983.193276,983.193291,983.193291
87,983.204369,983.207084,983.196005,983.203448,983.204354,983.204369,983.204369
88,983.212292,983.214987,983.203908,983.211371,983.212277,983.212292,983.212292


In [11]:
#@title Gradio
def my_plot(lat, col):
  fig, ax = plt.subplots()
  df.subtract(df[col], axis=0).plot(ax=ax)
  ax.legend(bbox_to_anchor=(1.01, 1))
  ax.set_ylabel('$\gamma_0$ diff (Gal)')
  ax.axvline(x=lat, c='k', lw=1)
  fig.tight_layout(pad=0)
  plt.close()
  tab = {key: f'{value:.6f} Gal' for key, value in dic_grav(lat).items()} 
  return fig, tab, 

iface = gr.Interface(
    fn=my_plot, 
    inputs=[gr.inputs.Slider(-90, +90 , 1, default=0, label='Latitud'),
            gr.inputs.Dropdown(list(df.columns), label='Modelo')],
    outputs=[gr.outputs.Image(type='plot', label='diff'),
             gr.outputs.KeyValues(label='Gravedad')],
    live=True,
    allow_flagging=False,
    allow_screenshot=True,
    title='Gravedad Teorica',
    description='Valores Teoricos de Gravedad',
    article = """<p style='text-align: center'>
    <a href='https://en.wikipedia.org/wiki/Theoretical_gravity'>
    Wikipedia | Theoretical Gravity</a></p>""",
    examples=[[6,'Formula Somigliana']],
    theme='huggingface', # "default", "compact" or "huggingface"
    layout='unaligned' # 'horizontal', 'unaligned', 'vertical'
    )

with io.capture_output() as captured:
    iface.launch(inline=True)
# print(iface.share_url)
IFrame(src=iface.share_url, width=1200, height=1000)

<IPython.core.display.Javascript object>